In [ ]:
pip install matplotlib numpy pillow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.animation import FuncAnimation, PillowWriter
import random

# --- PARÁMETROS ---
SIZE = 30
OBJETOS = 25
PASOS = 80  # Cuadros de la animación

VACIO, OBJETO, ROBOT = 0, 1, 2

# --- LÓGICA DEL ENTORNO ---
def crear_mapa(size, objetos):
    mapa = np.zeros((size, size), dtype=int)
    count = 0
    while count < objetos:
        x, y = random.randint(0, size-1), random.randint(0, size-1)
        if mapa[x, y] == VACIO:
            mapa[x, y] = OBJETO
            count += 1
    return mapa

def sensor_distancia(mapa, x, y, dx, dy):
    distancia = 0
    cx, cy = x, y
    while 0 <= cx+dx < SIZE and 0 <= cy+dy < SIZE:
        cx, cy = cx+dx, cy+dy
        distancia += 1
        if mapa[cx, cy] == OBJETO: break
    return distancia

def rotar(dx, dy, angulo):
    if angulo == 90: return dy, -dx
    if angulo == -90: return -dy, dx
    if angulo == 180: return -dx, -dy
    return dx, dy

# --- PREPARACIÓN DE LA ANIMACIÓN ---
mapa_estatico = crear_mapa(SIZE, OBJETOS)
estado = {
    'pos': [SIZE//2, SIZE//2],
    'dir': [0, 1]  # Derecha
}

fig, ax = plt.subplots(figsize=(6, 6))
cmap = ListedColormap(['#ecf0f1', '#2c3e50', '#e74c3c']) # Blanco, Gris, Rojo

# Dibujamos el primer frame
img = ax.imshow(mapa_estatico, cmap=cmap, vmin=0, vmax=2, animated=True)
ax.axis('off')

def actualizar(frame):
    rx, ry = estado['pos']
    dx, dy = estado['dir']

    # 1. Sensores
    frente = sensor_distancia(mapa_estatico, rx, ry, dx, dy)
    idx, idy = rotar(dx, dy, -90)
    izq = sensor_distancia(mapa_estatico, rx, ry, idx, idy)
    ddx, ddy = rotar(dx, dy, 90)
    der = sensor_distancia(mapa_estatico, rx, ry, ddx, ddy)

    # 2. Toma de decisiones
    if frente <= 2:
        if izq > der:
            estado['dir'] = [idx, idy]
        elif der > izq:
            estado['dir'] = [ddx, ddy]
        else:
            estado['dir'] = rotar(dx, dy, 180)

    # 3. Mover
    ndx, ndy = estado['dir']
    new_x, new_y = rx + ndx, ry + ndy

    if 0 <= new_x < SIZE and 0 <= new_y < SIZE and mapa_estatico[new_x, new_y] != OBJETO:
        estado['pos'] = [new_x, new_y]
    else:
        # Si choca de frente, rota aleatorio para el siguiente frame
        estado['dir'] = rotar(ndx, ndy, 90 if random.random() > 0.5 else -90)

    # 4. Actualizar Imagen
    frame_mapa = mapa_estatico.copy()
    frame_mapa[estado['pos'][0], estado['pos'][1]] = ROBOT
    img.set_array(frame_mapa)
    ax.set_title(f"Paso {frame} de {PASOS}")

    return [img]

# Crear objeto de animación
ani = FuncAnimation(fig, actualizar, frames=PASOS, interval=100, blit=True)

# --- GUARDAR EL ARCHIVO ---
print("Generando animación, por favor espera...")
writer = PillowWriter(fps=10)
ani.save("simulacion_robot.gif", writer=writer)
plt.close() # Cierra la figura para evitar errores de "0 Axes"

print("¡Listo! Busca el archivo 'simulacion_robot.gif' en tu carpeta.")